In [1]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import time

#### USING THE API TO FARM ON REDDIT

In [5]:
url_comms = "https://api.pushshift.io/reddit/search/comment"

In [6]:
params = {
    'subreddit' : 'vegan', 
    'size' : 500
}

In [7]:
res_vegan = requests.get(url_comms, params)

In [8]:
res_vegan.status_code

200

In [9]:
data = res_vegan.json()

In [10]:
posts_vegan = data['data']

In [11]:
len(posts_vegan)

100

In [12]:
#we can only get 100 post and we need a ton so we need to get the created utc value to use the before parameter to get different posts

In [13]:
posts_vegan[0]['created_utc']

1627486214

In [14]:
posts_vegan[99]['created_utc']

1627484064

In [16]:
#This function will get us the 20000 most recent posts
def farm(subreddit, n):
    
    df_list = []
    current_time = 1627486214 #the first post
    
    for _ in range(n):
        res = requests.get(
            url_comms,
            params={
                "subreddit": subreddit,
                "size": 500,
                "before": current_time
            }
        )
        time.sleep(3)
        df = pd.DataFrame(res.json()['data'])
        df = df.loc[:, ["subreddit", "body", "created_utc"]]
        df_list.append(df)
        current_time = df.created_utc.min()
        
    return pd.concat(df_list, axis=0)

In [17]:
df_vegan = farm('vegan', 200)

In [18]:
df_vegan.shape

(20000, 3)

In [19]:
#we have 20000 posts

In [20]:
df_vegan.sample(10)

,subreddit,body,created_utc
13,vegan,Dunno how much clearer i can make it lmao it w...,1627149049
51,vegan,[deleted],1627294174
87,vegan,Check out [Vegan Bootcamp](https://vbcamp.org/...,1627260433
96,vegan,Hard to face public pressure like he's certain...,1626913814
37,vegan,This is what I can interpret. Animals are forc...,1627366883
59,vegan,[removed],1626928221
16,vegan,"They're delicious, albeit bland and need seaso...",1627159229
55,vegan,"That's an HR situation, holy hell what a sick ...",1627192925
76,vegan,There's also bioavailability. Animals can dige...,1627423742
77,vegan,I made the dough July 7th and kept it in the f...,1627345875


In [22]:
params_1 = {
    'subreddit' : 'vegetarian', 
    'size' : 500
}
res_vegetarian = requests.get(url_comms, params_1)

In [23]:
res_vegetarian.status_code

200

In [24]:
data_1 = res_vegetarian.json()

In [25]:
posts_vegetarian = data_1['data']
len(posts_vegetarian)

100

In [27]:
posts_vegetarian[0]['created_utc']

1627487081

In [28]:
#This function will get us the 20000 most recent posts
def farm_vegetarian(subreddit, n):
    
    df_list = []
    current_time = 1627487081 #the first post
    
    for _ in range(n):
        res = requests.get(
            url_comms,
            params={
                "subreddit": subreddit,
                "size": 500,
                "before": current_time
            }
        )
        time.sleep(3)
        df = pd.DataFrame(res.json()['data'])
        df = df.loc[:, ["subreddit", "body", "created_utc"]]
        df_list.append(df)
        current_time = df.created_utc.min()
        
    return pd.concat(df_list, axis=0)

In [29]:
df_vegetarian = farm_vegetarian('vegetarian', 200)

In [34]:
df_vegetarian.shape

(20000, 4)

In [30]:
df_vegetarian.sample(10)

,subreddit,body,created_utc
28,vegetarian,"Hey, I hate tomatoes as well (unless it's part...",1626146196
54,vegetarian,Japanese curry washed down with cream soda!,1625682568
0,vegetarian,I use the CodeCheck app in the UK - Know it's ...,1624909111
43,vegetarian,Any tips to make the transition easier? I’ve b...,1625426441
91,vegetarian,"Alright, thank you. I usually do around 30 min...",1623961491
18,vegetarian,"Hi, this is a [really common question](https:/...",1621887953
25,vegetarian,I was almost anemic before going vegetarian. M...,1623762774
7,vegetarian,First picture made me think “is this a troll? ...,1625592208
31,vegetarian,[removed],1624750706
33,vegetarian,[deleted],1625747148


In [31]:
df_complete = pd.concat([df_vegan,df_vegetarian])

In [32]:
df_complete.sample(10)

,subreddit,body,created_utc
5,vegan,You're welcome!,1627167920
38,vegan,Nice to see a non-plastic alternative to leather,1627415667
86,vegetarian,Your first sentence made my night. I used to b...,1622595251
13,vegetarian,I make these rly nice sandwiches that I love w...,1626308603
46,vegan,The Ghostbusters music works for me,1627326209
93,vegetarian,I really don’t like the taste lol. I much pref...,1625618357
85,vegetarian,"I think I'm in a similar boat, after living wi...",1626211077
91,vegan,Talking about dogs the same way people talk ab...,1626749994
69,vegan,"I don't like to cook much, so I often eat a ve...",1627309972
50,vegetarian,"Oooh, I had not thought of that mod and it is ...",1624932174


In [33]:
#we send it to csv for EDA!

In [34]:
df_complete.to_csv('dirty_table.csv')